# PROJECT. Определение количества ботов на сайте
## ЗАДАЧИ:
- Оценить количество ботов, которые заходят на сайт, и их долю от всего трафика.
- Определить, с каких источников и в какое время суток чаще всего заходят роботы.

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import json 
import requests
from pprint import pprint

Для выгрузки отчёта нам необходимы следующие данные:
 - источники трафика по модели атрибуции «Последний значимый источник»;
 - суммарное количество визитов;
 - доля ботов в  метрике ym:s:robotPercentage;
 
 Данные сгруппируем по часам визита и по параметру ym:s:isRobot, который определяет, робот или человек заходил на сайт.
Зададим лимит в 100000 строк. Период анализа: с 1 сентября 2020 года по 30 сентября 2020 года.

Для начала посмотрим на общее количество ботов и их долю от всего трафика

In [2]:
# Задаем параметры, токен
params1 = {'metrics': 'ym:s:visits',
          'dimensions': 'ym:s:isRobot,ym:s:lastsignTrafficSource',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}
token = 'AQAAAABbwb7rAAeW_xVRvRo9EkQ0nKMRbM5fvHg'
headers = {'Authorization': 'OAuth ' + token}

In [3]:
# Делаем запрос response. Выполним его с помощью функции requests.get()
response_1 = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params1, headers=headers)
print (response_1.status_code)
metrika_data = response_1.json()
print(metrika_data['data'][0:2])

200
[{'dimensions': [{'name': 'People', 'id': 'no'}, {'icon_id': '2', 'icon_type': 'traffic-source', 'name': 'Search engine traffic', 'id': 'organic'}], 'metrics': [2595.0]}, {'dimensions': [{'name': 'Robots', 'id': 'yes'}, {'icon_id': '2', 'icon_type': 'traffic-source', 'name': 'Search engine traffic', 'id': 'organic'}], 'metrics': [739.0]}]


In [4]:
# Напишем функцию, чтобы избавится от вложенных словарей
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

In [5]:
# Преобразуем наши данные с помощью функции и запишем в переменную metrika_list_of_dicts
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:3])

[{'ym:s:isRobot': 'People',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:visits': 2595.0},
 {'ym:s:isRobot': 'Robots',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:visits': 739.0},
 {'ym:s:isRobot': 'People',
  'ym:s:lastsignTrafficSource': 'Direct traffic',
  'ym:s:visits': 580.0},
 {'ym:s:isRobot': 'Robots',
  'ym:s:lastsignTrafficSource': 'Direct traffic',
  'ym:s:visits': 177.0},
 {'ym:s:isRobot': 'People',
  'ym:s:lastsignTrafficSource': 'Link traffic',
  'ym:s:visits': 95.0}]


In [6]:
# Создадим датафрейм
metrika_df = pd.DataFrame(metrika_list_of_dicts)
print(metrika_df)

   ym:s:isRobot     ym:s:lastsignTrafficSource  ym:s:visits
0        People          Search engine traffic       2595.0
1        Robots          Search engine traffic        739.0
2        People                 Direct traffic        580.0
3        Robots                 Direct traffic        177.0
4        People                   Link traffic         95.0
5        People         Social network traffic         69.0
6        Robots                   Link traffic         31.0
7        Robots         Social network traffic         11.0
8        People               Internal traffic          7.0
9        People            Cached page traffic          5.0
10       Robots               Internal traffic          3.0
11       People  Recommendation system traffic          2.0
12       Robots            Cached page traffic          1.0


In [7]:
# Выведем информацию о датафрейме
print(metrika_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ym:s:isRobot                13 non-null     object 
 1   ym:s:lastsignTrafficSource  13 non-null     object 
 2   ym:s:visits                 13 non-null     float64
dtypes: float64(1), object(2)
memory usage: 440.0+ bytes
None


Датафрейм состоит из трех колонок, пропущенных значений нет.

In [8]:
# Оценим общее количество ботов и их долю от всего трафика
all_visits = pd.DataFrame(metrika_df.groupby('ym:s:isRobot')['ym:s:visits'].sum())
all_visits['Rate_%'] = round(all_visits['ym:s:visits']/all_visits['ym:s:visits'].sum()*100, 2)
all_visits.columns=['Visits','Rate_%']
print(all_visits)

              Visits  Rate_%
ym:s:isRobot                
People        3353.0   77.71
Robots         962.0   22.29


Боты составляют 22.29 % от общего трафика (962 визита)

Далее детально рассмотрим долю ботов в разрезе источников трафика и времени посещения

In [9]:
# Задаем параметры
params_2 = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions': 'ym:s:lastsignTrafficSource,ym:s:hour',  
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}

In [10]:
# Делаем запрос response. Выполним его с помощью функции requests.get()
response_2 = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params_2, headers=headers)
print (response_2.status_code)
metrika_data_2 = response_2.json()
pprint(metrika_data_2['data'][0:2])

200
[{'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': '16', 'name': '16:00'}],
  'metrics': [251.0, 24.70119522]},
 {'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': '15', 'name': '15:00'}],
  'metrics': [240.0, 26.25]}]


In [11]:
# Преобразуем наши данные с помощью  функции и запишем в переменную metrika_list_of_dicts_2
metrika_list_of_dicts_2 = getMetrikaDataInListOfDicts(metrika_data_2)
pprint(metrika_list_of_dicts_2[0:3])

[{'ym:s:hour': '16:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 24.70119522,
  'ym:s:visits': 251.0},
 {'ym:s:hour': '15:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 26.25,
  'ym:s:visits': 240.0},
 {'ym:s:hour': '11:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 25.87719298,
  'ym:s:visits': 228.0}]


In [12]:
# Создаем датафрейм
metrika_df_2 = pd.DataFrame(metrika_list_of_dicts_2)
print(metrika_df_2)

        ym:s:lastsignTrafficSource ym:s:hour  ym:s:visits  \
0            Search engine traffic     16:00        251.0   
1            Search engine traffic     15:00        240.0   
2            Search engine traffic     11:00        228.0   
3            Search engine traffic     13:00        227.0   
4            Search engine traffic     12:00        225.0   
..                             ...       ...          ...   
102         Social network traffic     02:00          1.0   
103         Social network traffic     04:00          1.0   
104         Social network traffic     10:00          1.0   
105  Recommendation system traffic     08:00          1.0   
106  Recommendation system traffic     14:00          1.0   

     ym:s:robotPercentage  
0               24.701195  
1               26.250000  
2               25.877193  
3               28.193833  
4               24.888889  
..                    ...  
102              0.000000  
103              0.000000  
104            

In [13]:
# Переименуем столбцы и выведем информацию о датафрейме
metrika_df_2.columns=['Source','Hour','Visits','BotRate']
print(metrika_df_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Source   107 non-null    object 
 1   Hour     107 non-null    object 
 2   Visits   107 non-null    float64
 3   BotRate  107 non-null    float64
dtypes: float64(2), object(2)
memory usage: 3.5+ KB
None


Датафрейм состоит из четырех столбцов, пропущенных значений нет


In [14]:
# Выведем описательную статистику 
print(metrika_df_2.describe(include=['object']))
print('*************************************')
print(metrika_df_2.describe())


                       Source   Hour
count                     107    107
unique                      7     24
top     Search engine traffic  08:00
freq                       24      7
*************************************
           Visits     BotRate
count  107.000000  107.000000
mean    40.327103   19.817892
std     65.694659   18.885087
min      1.000000    0.000000
25%      2.000000    0.000000
50%      9.000000   19.565217
75%     41.000000   26.063596
max    251.000000  100.000000


Как видим из полученной статистики, чаще всего боты приходят из источника  Search engine traffic (частота 24, то есть каждый час). Самое частое время посещения ботами - 8 часов утра.
Максимальное количество посещений ботами за один час составляет 251, а их максимальная доля от общего трафика достигает 100 %
В среднем же на час приходится 40.33 посещения ботами, средний процент от общего трафика - 19.82

In [15]:
# Определим источники, из  которых чаще всего приходят боты
print('Источники в зависимости от частоты визитов ботов: ')
print(metrika_df_2['Source'].value_counts())
print('*************************************')
print('Источники в зависимости от средней доли ботов от общего количества визитов')
print(metrika_df_2.groupby('Source')['BotRate'].mean().sort_values(ascending=False))

Источники в зависимости от частоты визитов ботов: 
Search engine traffic            24
Direct traffic                   24
Link traffic                     22
Social network traffic           22
Internal traffic                  8
Cached page traffic               5
Recommendation system traffic     2
Name: Source, dtype: int64
*************************************
Источники в зависимости от средней доли ботов от общего количества визитов
Source
Link traffic                     27.249606
Internal traffic                 25.000000
Direct traffic                   23.835393
Search engine traffic            18.963520
Social network traffic           11.084055
Cached page traffic              10.000000
Recommendation system traffic     0.000000
Name: BotRate, dtype: float64


In [16]:
print('Максимальные показатели доли ботов в зависимости от источника')
print(metrika_df_2.groupby('Source')['BotRate'].max().sort_values(ascending=False))

Максимальные показатели доли ботов в зависимости от источника
Source
Internal traffic                 100.000000
Link traffic                     100.000000
Cached page traffic               50.000000
Social network traffic            50.000000
Direct traffic                    40.625000
Search engine traffic             28.193833
Recommendation system traffic      0.000000
Name: BotRate, dtype: float64


Чаще всего боты приходят из источников Search engine traffic, Direct traffic, Link traffic, Social network traffic
Наибольшее среднее процентное отношение ботов к общему количеству визитов у источников Link traffic, Internal traffic, Direct traffic.
Для определения источников, из которых приходит больше всего ботов, было взято среднее значение, так как оценка по максимальному процентному отношению была бы менее информативной. Например, 100 % ботов пришло из источника Internal traffic, но всего визитов ботов из этого источника было лишь 3.

In [17]:
# Создадим объединенный датафрейм из двух (количества источников, из которых приходят боты, по часам (count_df) и средней доли ботов по часам (rate_df))
count_df = pd.DataFrame(metrika_df_2.groupby('Hour')['BotRate'].count())
rate_df =  pd.DataFrame(metrika_df_2.groupby('Hour')['BotRate'].mean())
total_df = count_df.merge(rate_df, on = 'Hour').sort_values('BotRate_y', ascending=False)
total_df.columns = ['Source_count', 'BotRate_mean']
print(total_df)

       Source_count  BotRate_mean
Hour                             
03:00             3     44.312169
16:00             5     43.035477
22:00             4     34.579012
13:00             4     27.918994
09:00             5     26.106668
12:00             6     25.165701
10:00             4     25.071268
23:00             4     24.107407
06:00             4     21.524064
15:00             4     18.554370
01:00             5     18.018868
07:00             4     17.371297
20:00             4     17.165337
18:00             4     16.560885
04:00             3     15.458937
08:00             7     14.971301
11:00             6     14.556498
14:00             6     14.552707
21:00             4     13.624545
00:00             3     12.222222
19:00             4     12.150621
17:00             6     11.565452
05:00             3      8.000000
02:00             5      6.075388


Чаще всего (из всех 7 источников трафика) боты приходят в 8 часов утра. Далее по частоте (6 из 7 источников) боты приходят в 11, 12, 14 и 17 часов. Наименьшая частота наблюдается в полночь, а также в период с 3 до 5 утра.
Наибольший средний показатель доли ботов от общего количества посещений приходится на 3 часа утра и 16 часов дня.
Также большое количество (в процентном отношении) приходит в первой половине дня и поздно вечером. 

Рассмотрим показатели частоты посещений и доли ботов в разрезе времени суток.

In [18]:
# Преобразуем столбец с часами посещения в Тип datetime и выделим часы
metrika_df_2['Hour'] = pd.to_datetime(metrika_df_2['Hour'])
metrika_df_2['Hour'] = metrika_df_2['Hour'].dt.hour
print(metrika_df_2['Hour'])

0      16
1      15
2      11
3      13
4      12
       ..
102     2
103     4
104    10
105     8
106    14
Name: Hour, Length: 107, dtype: int64


In [19]:
# Напишем функцию для определения времени суток
def get_time_of_day(time):
    if 0 <= time <= 6:
        return 'night'
    elif 6 < time <= 12:
        return 'morning'
    elif 12 < time <= 18:
        return 'day'
    elif 18 < time <= 23:
        return 'evening'
    else:
        return 'else'
metrika_df_2['time_of_day'] = metrika_df_2['Hour'].apply(get_time_of_day)

In [20]:
print(metrika_df_2.head())

                  Source  Hour  Visits    BotRate time_of_day
0  Search engine traffic    16   251.0  24.701195         day
1  Search engine traffic    15   240.0  26.250000         day
2  Search engine traffic    11   228.0  25.877193     morning
3  Search engine traffic    13   227.0  28.193833         day
4  Search engine traffic    12   225.0  24.888889     morning


In [24]:
print('Частота посещения сайта ботами в зависимости от времени суток:')
print(metrika_df_2.groupby('time_of_day')['Source'].count().sort_values(ascending=False))
print('***********************')
print('Средняя доля ботов от всех визитов в зависимости от времени суток, %:')
print(round(metrika_df_2.groupby('time_of_day')['BotRate'].mean(), 2))

Частота посещения сайта ботами в зависимости от времени суток:
time_of_day
morning    32
day        29
night      26
evening    20
Name: Source, dtype: int64
***********************
Средняя доля ботов от всех визитов в зависимости от времени суток, %:
time_of_day
day        21.52
evening    20.33
morning    20.11
night      17.17
Name: BotRate, dtype: float64


Чаще всего боты приходят на сайт утром и днем. Меньше всего - вечером, при этом средняя доля ботов у вечерних посещений на втором месте.
У дневных посещений и утренних средняя доля ботов примерно одинаковая. Ночью самый низкий средний показатель доли ботов в общем трафике.

# Выводы:
На основании проведенного анализа можно сделать следующие выводы:
- боты составляют 22.29 % от общего трафика (962 визита);
- чаще всего боты приходят из источника  Search engine traffic и Direct traffic (частота 24, то есть каждый час);
- самое частое время посещения ботами - 8 часов утра;
- чаще всего боты приходят в утреннее время (32 наблюдения);
- средняя доля ботов от общего трафика выше у дневных посещений (21.52 %)